In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np

# add the path to the abcTau package
import sys
#sys.path.append('./abcTau')
sys.path.append('C:\\Users\\ipochino\\.conda\\envs\\isttc\\Lib\\site-packages\\abcTau') # IP: replaced previous line with that; relative path was not working
import abcTau

### Generate time series

In [2]:
save_folder = 'Q:\\Personal\\Irina\\projects\\isttc\\results\\synthetic_data\\test_full_split_parametric\\dataset\\'

n_signals = 100

In [3]:
tau = np.array([100])
D = 1/tau
deltaT = 1
T = 100*1000
numTrials = 1

data_mean = 0.01 # average of firing rate
data_var = 0.02 # variance of firing rate

binSize = 50 # bins size for binning the data for calculating acf
binsData =  np.arange(0, T + binSize, binSize)
numBinData = len(binsData)-1
#print(binsData)
print(numBinData)

num_lags = 20
maxTimeLag = 1000 

2000


In [10]:
for signal_ in range(n_signals):
    ###############
    # OU process 
    ou = abcTau.OU_gen(tau, D, deltaT, T, numTrials)
    print('ou.shape: ', ou.shape)
    
    ou_check = np.max(ou)
    print(ou_check)
    # adjust OU process - scale, shift 
    ou_std = np.sqrt(data_var)
    ou_mean = data_mean
    ou_all = ou_std * ou + ou_mean # this is scale and shift, where is rectify? - no rectify here, this will come for the Poisson rate
    print('ou_all.shape: ', ou_all.shape)
    
    ###############
    # One tau UO with Poisson spike count
    ou_std =  np.sqrt(data_var - data_mean)/deltaT # law of total variance  
    ou_mean = data_mean/deltaT # law of total expectation
    
    # fit mean and var
    ou_poisson = ou_std * ou + ou_mean
    ou_poisson[ou_poisson < 0] = 0 # rectifying
    
    # bin rate and generate spikes
    numBin = int(T/deltaT)
    ou_poisson_rate_ = abcTau.binData(ou_poisson, [numTrials,numBin]) * deltaT
    ou_poisson_rate = np.random.poisson(ou_poisson_rate_)
    
    ###############
    # Going from rate to spikes
    spikeTrain = np.zeros(ou_poisson_rate.shape)
    print(spikeTrain.shape)
    
    for i in range(ou_poisson_rate.shape[0]):
        spikeTrain[i,:] = [1 if ou_poisson_rate[i,j] > np.random.random() else 0 for j in range(ou_poisson_rate.shape[1])]
    
    spike_times = np.where(np.squeeze(spikeTrain) == 1)[0]
    print('N spikes {}'.format(spike_times.shape))
    
    # bin for ACF calculation
    ou_spiketrain_binned = abcTau.binData(spikeTrain, [numTrials, numBinData]) * deltaT
    print(ou_spiketrain_binned.shape)
    
    # store
    np.save(save_folder + 'spike_times_' + str(signal_) + '.npy', spike_times)
    np.save(save_folder + 'spike_times_binned_' + str(signal_) + '.npy', ou_spiketrain_binned)

ou.shape:  (1, 100000)
3.7303984395255494
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (3928,)
(1, 2000)
ou.shape:  (1, 100000)
3.7729011017426792
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4250,)
(1, 2000)
ou.shape:  (1, 100000)
3.855612136800286
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4116,)
(1, 2000)
ou.shape:  (1, 100000)
4.2342473663336415
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4504,)
(1, 2000)
ou.shape:  (1, 100000)
3.477862858067011
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4142,)
(1, 2000)
ou.shape:  (1, 100000)
4.215966391799468
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4506,)
(1, 2000)
ou.shape:  (1, 100000)
4.086189371775626
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4294,)
(1, 2000)
ou.shape:  (1, 100000)
3.982184734638709
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4168,)
(1, 2000)
ou.shape:  (1, 100000)
3.447985403726402
ou_all.shape:  (1, 100000)
(1, 100000)
N spikes (4006,)
(1, 2000)
ou.shape:  (1, 100000)
4.296465852